In [6]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [7]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os
import base64

In [8]:
def derive_key(password: str, salt: bytes, iterations: int = 100_000) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 256-bit key
        salt=salt,
        iterations=iterations,
        backend=default_backend()
    )
    return kdf.derive(password.encode())


In [9]:
def encrypt(plaintext: str, password: str):
    salt = os.urandom(16)  # Random salt
    iv = os.urandom(16)    # Initialization vector
    key = derive_key(password, salt)

    # Pad the plaintext to a multiple of 16 bytes
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    # AES encryption in CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return {
        'ciphertext': base64.b64encode(ciphertext).decode(),
        'salt': base64.b64encode(salt).decode(),
        'iv': base64.b64encode(iv).decode()
    }


In [10]:
def decrypt(ciphertext_b64: str, password: str, salt_b64: str, iv_b64: str):
    salt = base64.b64decode(salt_b64)
    iv = base64.b64decode(iv_b64)
    ciphertext = base64.b64decode(ciphertext_b64)

    key = derive_key(password, salt)

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    # Remove PKCS7 padding
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()

    return plaintext.decode()


In [11]:
# Demo usage
password = "mypassword123"
secret_message = "Dear diary, today I had coffee and thought about the universe."

# Encrypt
encrypted = encrypt(secret_message, password)
print("Encrypted:")
print(encrypted)

# Decrypt
decrypted = decrypt(encrypted['ciphertext'], password, encrypted['salt'], encrypted['iv'])
print("\nDecrypted:")
print(decrypted)


Encrypted:
{'ciphertext': 'ta5dOwwFXpd31vvwuM7zjtDkuSnQTUz2CVgTglXI05CmAYe+/juRcEMEfgPiY/Zs+xKrL7mHucGJGmU+sdpfZg==', 'salt': 'T0beeCPVzItsCtFLte7xIg==', 'iv': 'QKHEv0ESibkjTa2rEJ36aQ=='}

Decrypted:
Dear diary, today I had coffee and thought about the universe.
